In [2]:
import importlib
import torch
import torchvision


from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from pathlib import Path
from augs import AugTransform

import time
import learning_loop

from learning_loop import BATCH_SIZE

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [4]:
import models

BarlowTwins = models.BauglowTwins

In [10]:
!nvidia-smi

Fri Dec 23 00:18:44 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   30C    P0    23W / 300W |      2MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
def reload():
    importlib.reload(learning_loop)
    importlib.reload(models)

    global BATCH_SIZE, BarlowTwins, a_loop

    a_loop = learning_loop.aug_learning_loop
    BATCH_SIZE = learning_loop.BATCH_SIZE
    BarlowTwins = models.BauglowTwins

In [7]:
stl = datasets.STL10(
    root="data",
    split="train",
    download=True,
    transform=AugTransform()
)

/opt/conda/envs/pytorch/lib/python3.9/site-packages/torchvision/transforms/transforms.py:899: UserWarning: Argument 'interpolation' of type int is deprecated since 0.13 and will be removed in 0.15. Please use InterpolationMode enum.
  warnings.warn(


Files already downloaded and verified


In [9]:
stl_loader = DataLoader(stl, batch_size=BATCH_SIZE)
data, cls = next(iter(stl_loader))

In [11]:
reload()
model = BarlowTwins(BATCH_SIZE, use_sqrt=True).to(device)

In [14]:
torch.cuda.empty_cache()

In [15]:
!nvidia-smi

Fri Dec 23 00:20:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  On   | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0    46W / 300W |  15969MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [13]:
x, y = data
l = model(x.cuda(), y.cuda())

OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 15.78 GiB total capacity; 14.16 GiB already allocated; 190.94 MiB free; 14.47 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [16]:
reload()

a_loop(
    model,
    dataset=stl,
    output_dir="aug",
    delete_existing_dir=True
)

/opt/conda/envs/pytorch/lib/python3.9/site-packages/ipykernel_launcher.py -f /home/ubuntu/.local/share/jupyter/runtime/kernel-c279bef5-f40c-42e3-9976-0ee6d9a1fc8b.json


OutOfMemoryError: CUDA out of memory. Tried to allocate 784.00 MiB (GPU 0; 15.78 GiB total capacity; 14.45 GiB already allocated; 42.94 MiB free; 14.61 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF